# University Management System

In [ ]:
# Import Dependencies
import mysql.connector
import re
import pandas as pd

In [ ]:
# Fetch database url form config.txt
c = [re.findall(r".*='(.*)'.?", line) for line in open('.\config.txt')]
h = c[0][0]
u = c[1][0]
p = c[2][0]

In [ ]:
# Establish database connection
mydb = mysql.connector.connect(host=h,username=u,password=p,database="ums")
mycursor = mydb.cursor()

In [ ]:
# Function which returns names of all the columns in input table if exists
def getColumnNames(tableName):
    query = "DESCRIBE {};".format(tableName)
    mycursor.execute(query)
    columns = []
    for column in list(mycursor.fetchall()):
        columns.append(column[0])
    return(columns)

# 1) Change column name from Date_of_Birth to DOB in students Table

In [ ]:
mycursor.execute("ALTER TABLE STUDENTS CHANGE Date_of_Birth DOB DATE;")

# 2) Fetch all the values from students table

In [ ]:
mycursor.execute("SELECT * FROM STUDENTS;")
result = mycursor.fetchall()
columns = getColumnNames('STUDENTS')
display(pd.DataFrame(data=result,columns=columns))

# 3) Display students whose CPI is more than 8 and gender is male

In [ ]:
mycursor.execute("SELECT * FROM STUDENTS WHERE CPI > 8 AND GENDER = 'M';")
result = mycursor.fetchall()
columns = getColumnNames('STUDENTS')
display(pd.DataFrame(data=result,columns=columns))

# 4) Display Infromation of students and courses from report card

In [ ]:
mycursor.execute("SELECT STUDENTS.STUDENT_ID, CONCAT(F_NAME, ' ', L_NAME) AS S_NAME, C_Name, Midterm_Marks, Endterm_Marks FROM STUDENTS NATURAL JOIN REPORT_CARD LEFT JOIN COURSES ON REPORT_CARD.Course_ID = COURSES.Course_ID;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['STUDENT_ID','S_NAME','C_NAME','MIDTERM_MARKS','ENDTERM_MARKS']))

# 5) Display all the values stored in Employees table

In [ ]:
mycursor.execute("SELECT * FROM EMPLOYEES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=getColumnNames('EMPLOYEES')))

# 6) Display faculty details

In [ ]:
mycursor.execute("SELECT EMPLOYEES.Emp_ID,Emp_Name,Gender,Experience FROM EMPLOYEES NATURAL JOIN FACULTIES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Emp_ID','Emp_Name','Gender','Experience(YEARS)']))

# 7) Display details of faculties who are also trainers

In [ ]:
mycursor.execute("SELECT EMPLOYEES.Emp_ID,Emp_Name,Gender,Salary,Specialization FROM EMPLOYEES NATURAL JOIN FACULTIES INNER JOIN TRAINERS ON FACULTIES.Emp_ID=TRAINERS.Emp_ID;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Emp_ID','Emp_Name','Gender','Salary','Specialization']))

# 8) Create view named Scholarship which contains S_ID, names and CPI of students with CPI > 8

In [ ]:
mycursor.execute("CREATE OR REPLACE VIEW SCHOLARSHIP AS SELECT Student_ID,CONCAT(F_NAME,' ',L_NAME),CPI FROM STUDENTS WHERE CPI >= 8;")
mycursor.execute("SELECT * FROM SCHOLARSHIP;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Student_ID','S_NAME','CPI']))

# 9) Find average CPI

In [ ]:
mycursor.execute("SELECT ROUND(AVG(CPI),2) FROM STUDENTS;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['AVG(CPI)']))

# 10) Find average salary of Faculties

In [ ]:
mycursor.execute("SELECT ROUND(AVG(SALARY),2) FROM FACULTIES NATURAL JOIN EMPLOYEES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['AVG(Salary)']))

# 11) Find average salary of Trainers

In [ ]:
mycursor.execute("SELECT ROUND(AVG(SALARY),2) FROM TRAINERS NATURAL JOIN EMPLOYEES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['AVG(Salary)']))

# 12) Find average salary of Staffs

In [ ]:
mycursor.execute("SELECT ROUND(AVG(SALARY),2) FROM STAFFS NATURAL JOIN EMPLOYEES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['AVG(Salary)']))

# 13) Update email of Student named Jay

In [ ]:
mycursor.execute("UPDATE STUDENTS SET Email='jay@anyway.com' WHERE F_NAME = 'Jay';")

# 14) Add a column named type in staffs

In [ ]:
mycursor.execute("ALTER TABLE STAFFS ADD COLUMN Type VARCHAR(255) DEFAULT 'Lease';")
mycursor.execute("SELECT * FROM STAFFS;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=getColumnNames('STAFFS')))

# 15) Display Employees details with department names in which they work

In [ ]:
mycursor.execute("SELECT Emp_ID,Emp_NAME,Gender,DEPT_NAME FROM EMPLOYEES LEFT JOIN DEPARTMENTS ON DEPT_ID = Department;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Emp_ID','Emp_NAME','Gender','Department']))

# 16) Insert few values in Joined relation

In [ ]:
mycursor.execute("INSERT INTO JOINED VALUES (456,1235),(456,3456),(345,9879),(567,3456),(567,9879),(567,5674),(234,7651),(678,8762),(789,8762),(987,5674);")

# 17) Display details of clubs along with total number of students joined

In [ ]:
mycursor.execute("SELECT CLUBS.Club_ID,Club_Name,Reg_Fees,COUNT(DISTINCT Student_ID) FROM CLUBS LEFT JOIN JOINED ON JOINED.Club_ID=CLUBS.Club_ID GROUP BY Club_ID;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Club_ID','Club_Name','Reg_Fees','Enrollment']))

# 18) Display structure of Employees Table

In [ ]:
mycursor.execute("DESCRIBE EMPLOYEES;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['Field','Type','Null','Key','Default','Extra']))

# 19) Display information of students who didn't took part in any club

In [ ]:
mycursor.execute("SELECT * FROM STUDENTS WHERE STUDENT_ID NOT IN (SELECT DISTINCT STUDENT_ID FROM joined);")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=getColumnNames('STUDENTS')))

# 20) Delete information of students who have CPI less than 7.0 because they are expelled from university

In [ ]:
mycursor.execute("DELETE FROM STUDENTS WHERE CPI < 7.0;")

# 21) Display information who joined more than 1 clubs along with club counts

In [ ]:
mycursor.execute("SELECT CONCAT(F_NAME,' ',L_NAME) AS S_Name,COUNT(*) AS Club_Count FROM JOINED NATURAL JOIN STUDENTS GROUP BY Student_ID HAVING Club_Count > 1;")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=['S_Name','Club_Count']))

# 22) Display information of courses starting with 'C'

In [ ]:
mycursor.execute("SELECT * FROM COURSES WHERE C_Name LIKE 'C%';")
result = mycursor.fetchall()
display(pd.DataFrame(data=result,columns=getColumnNames('COURSES')))

# 23) Truncate table Staffs

In [ ]:
mycursor.execute("TRUNCATE TABLE STAFFS;")

# 24) Delete table Staffs

In [ ]:
mycursor.execute("DROP TABLE STAFFS;")

# 25) Drop schema ums

In [ ]:
mycursor.execute("DROP SCHEMA UMS;")